# 0: Import packages

In [ ]:
from PIL import Image, ImageFile #pip install Pillow==9.4.0
import sys
import os
import numpy as np
import logging
import glob
import subprocess
import json
import pandas as pd

from ImageCrop import ImagePreprocessor
from SpotterWrapper import Spotter, PolygonVisualizer
from IPython.display import display
from shapely.geometry import Polygon

logging.basicConfig(level=logging.INFO)
Image.MAX_IMAGE_PIXELS=None
ImageFile.LOAD_TRUNCATED_IMAGES = True


# 1: Specify filepaths

In [ ]:
# Name folders for raw data and processed data
map_data_topfolder = 'raw_maps_20231024'
map_strec_topfolder = 'processed/strec'

for fp in [map_strec_topfolder]:
    if not os.path.isdir(fp):
        os.makedirs(fp)

# IMPORTANT! Locate spotter directory and detectron weights
git_clone_location = 'C:/repo/'
spotter_directory = git_clone_location + 'mapkurator-spotter/spotter-v2'
model_weights = git_clone_location + 'detectron2-master/detectron2/checkpoint/model_v2_en.pth'
spotter_config = spotter_directory + '/configs/PALEJUN/Finetune/Rumsey_Polygon_Finetune.yaml'

# 2: Crop all jpeg maps in (user defined) map_data_topfolder

In [ ]:
def pyramid_scan(img_path, output_dir, save_each_layer=False):
    image = Image.open(img_path)
    image_preprocessor = ImagePreprocessor(image, overlapping_tolerance=0.3, num_layers=5, min_patch_resolution=512, max_patch_resolution=4096)
    image_preprocessor.process()
    print("preprocessing done")
    spotter = Spotter(spotter_config, model_weights, confidence_thresh=0.7, draw_thresh=0.85)
    all_layer_results = []

    base_image_batch, base_offset_xs, base_offset_ys = image_preprocessor.get_image_patches(0)
    vis = PolygonVisualizer()
    vis.canvas_from_patches(base_image_batch, base_offset_xs, base_offset_ys)

    for i in range(image_preprocessor.num_layers):
        # If you want to save for each layer, uncomment the following line
        # image_preprocessor.save_patches(os.path.join(output_dir, f'layer_{i}_patches'), layer=i)

        image_batch, offset_xs, offset_ys = image_preprocessor.get_image_patches(i)
        spotter.load_batch(image_batch, offset_xs, offset_ys)
        results = spotter.inference_batch()
        all_layer_results.extend(results)

        #all_layer_offset_xs.extend(offset_xs)
        #all_layer_offset_ys.extend(offset_ys)

        if save_each_layer == True:
            vis.draw(results).save(os.path.join(output_dir, f'combined_tagged_{i}.png'))
            vis.save_json(results, os.path.join(output_dir, f'combined_tagged_{i}.json'))
        else:
            pass

    vis.draw(all_layer_results).save(os.path.join(output_dir, f'combined_tagged_all_layers.png'))
    vis.save_json(all_layer_results, os.path.join(output_dir, f'combined_tagged_all_layers.json'))

# Run crop on all maps
for map_data_subfolder in next(os.walk(map_data_topfolder))[1]:
    jpeg_list = glob.glob(map_data_topfolder + '/' + map_data_subfolder + '/*.jpeg')
    if len(jpeg_list) != 1:
        print(map_data_subfolder + " failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.")
    else:
        map_image = jpeg_list[0].split("\\")[1]
        if map_data_subfolder in ['1846_vandevelde', '1874_saunders', '1845_kiepert']: # '1858_vandevelde', '1874_saunders', '1845_kiepert']: #,,]: #'1858_vandevelde', '1847_tobler', '1845_kiepert'
            img_path = map_data_topfolder + '/' + map_data_subfolder + "/" + map_image
            map_name = os.path.basename(img_path).split('.')[0] # get the map name without extension
            output_dir = os.path.join(map_strec_topfolder, map_name)
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            pyramid_scan(img_path, output_dir, save_each_layer=False)
            logging.info('Done cropping %s' %img_path )

# 3: Label Combination

In [2]:
from PIL import Image, ImageFile
import json 
import pandas as pd
from collections import Counter
from shapely.geometry import Polygon, MultiPolygon
from itertools import combinations

import numpy as np
import importlib
import Clustering
import TextRectify
import TextAmalgamate
import ExtractHandling
import json
import pickle
import SpotterWrapper
import Grouping
import BezierSplineMetric
import FontSimilarity
import SequenceRecovery
import SubwordDeduplication as sd
import NestedWordFlattening as nwf
import RumseyMetric
import SingleLineSequenceRecovery as slsr
importlib.reload(SpotterWrapper)
importlib.reload(Grouping)
importlib.reload(Clustering)
importlib.reload(TextRectify)
importlib.reload(TextAmalgamate)
importlib.reload(ExtractHandling)
importlib.reload(BezierSplineMetric)
importlib.reload(FontSimilarity)
importlib.reload(SequenceRecovery)

map_name_in_strec = 'vandevelde_1846' # 'kiepert_1845', 'saunders_1874', 'vandevelde_1846'

Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda


## 3.1 Subword Deduplication

In [8]:
sd.subword_deduplication(map_name_in_strec, do_cluster_pre_merge=True)

971 polygons kept.


## 3.2 Nested Word Flattening

In [2]:
def testing_vis(df, map_name_in_strec, suffix):
    polygons = [el[0] for el in df['labels']]
    texts = [el[1] for el in df['labels']]
    vis = SpotterWrapper.PolygonVisualizer()
    canvas = Image.open(f'processed/strec/{map_name_in_strec}/raw.jpeg')
    vis.canvas_from_image(canvas)
    vis.draw_poly(polygons, texts, PCA_feature_list=None, BSplines=None, random_color=True)
    vis.save(f'processed/strec/{map_name_in_strec}/testing_{suffix}.jpeg')

In [11]:
# Amalgamation stage - assumes there exists "combined_tagged_all_layers_rectified_premerge.json" in map_name_in_strec processed folder.
df = ExtractHandling.prepare_labels_for_amalgamation(map_name_in_strec)
df_orig = TextAmalgamate.amalgamate_labels_wrapper(df, 0.75, .5)
# Save amalgamated labels
with open(f'processed/strec/{map_name_in_strec}/deduplicated_flattened_labels.pickle', 'wb') as handle:
    pickle.dump(df_orig, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Save visualization
testing_vis(df_orig, map_name_in_strec, "orig_nwf")

971 labels.
893 labels.
870 labels.
867 labels.
866 labels.
Amalgamation completed with 866 labels.


In [ ]:
# Amalgamation stage - assumes there exists "combined_tagged_all_layers_rectified_premerge.json" in map_name_in_strec processed folder.
df = ExtractHandling.prepare_labels_for_amalgamation(map_name_in_strec)
df_new = pd.DataFrame({"labels": nwf.nwf_wrapper(df['labels'].tolist(), 0.75, 0.5)})
# Save amalgamated labels
with open(f'processed/strec/{map_name_in_strec}/deduplicated_flattened_labels.pickle', 'wb') as handle:
    pickle.dump(df_new, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Save visualization
testing_vis(df_new, map_name_in_strec, "new_nwf")

## 3.3 Multi-Word Sequence Recovery

### 3.3.1 Prepare by calculating spline and font metrics

In [13]:
df = pickle.load(open('processed/strec/' + map_name_in_strec + '/deduplicated_flattened_labels.pickle', 'rb'))
df['polygons'] = df['labels'].apply(lambda x: x[0])
df['texts'] = df['labels'].apply(lambda x: x[1])

# Uncomment to draw splines later
## BezierSplineMetric.draw_splines(map_name_in_strec, polygons, texts, PCA_features, all_splines)

# reset index so list-based operations match df index
df = df.reset_index(drop=True).copy()

# pca for principal directions
df['PCA_features'] = Grouping.calc_PCA_feats(df['polygons'], do_separation=True, enhance_coords=True)

# find neighbors for spline and font metric consideration
df = BezierSplineMetric.calc_neighbours(df, radius_multiplier = 40)

# calculate spline metric between identified neighbors
df = BezierSplineMetric.spline_metric(df)

# calculate font metric between identified neighbors - long due to need to work with images
df = FontSimilarity.calc_font_similarities(df, map_name_in_strec)

# calculate rumsey metric for combination
df = RumseyMetric.calc_rumsey_metric(df)

with open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Optional - draw splines
#df = pickle.load(open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'rb'))
#BezierSplineMetric.draw_splines(map_name_in_strec, df['polygons'].tolist(), df['texts'].tolist(), df['PCA_features'].tolist(), df['all_splines'].explode().dropna().tolist(), spline_metric_threshold = 0.01)

### 3.3.2 Iterative Sequence Recovery

#### A. Rumsey's Method

In [15]:
# load in seq rec prepared
df = pickle.load(open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'rb'))
## Drop PCA_features - no longer needed, makes me feel good to discard stuff i don't need
df.drop('PCA_features', axis=1, inplace=True)
df.drop('all_splines', axis=1, inplace=True)

# calculate using rumsey metric
df = SequenceRecovery.sl_sequence_recovery_wrapper(df, use_rumsey_metric=True)

# map to dataframe
new_texts = []
new_labels = []
for index, row in df.iterrows():
    sorted_text = sorted(row['text_list'], key=lambda x: x[0][0])
    new_texts.append(" ".join([_text[1] for _text in sorted_text]))
    new_labels.append((row['labels'][0], row['texts']))
df['labels'] = new_labels
df['texts'] = new_texts

# save
with open(f'processed/strec/{map_name_in_strec}/fully_processed_labels_rumsey.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

865 labels.
584 labels.
Sequence Recovery completed with 584 labels.


#### B. Our Method - Old, Issues with Two Recovered Sequences Combining

In [2]:
# load in seq rec prepared
df = pickle.load(open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'rb'))
## Drop PCA_features - no longer needed, makes me feel good to discard stuff i don't need
df.drop('PCA_features', axis=1, inplace=True)
df.drop('all_splines', axis=1, inplace=True)

# calculate using our metric
df = SequenceRecovery.sl_sequence_recovery_wrapper(df, font_threshold=.5, bezier_threshold=1.5, use_rumsey_metric=False)

# map to dataframe
new_texts = []
new_labels = []
for index, row in df.iterrows():
    sorted_text = sorted(row['text_list'], key=lambda x: x[0][0])
    new_texts.append(" ".join([_text[1] for _text in sorted_text]))
    new_labels.append((row['labels'][0], row['texts']))

df['labels'] = new_labels
df['texts'] = new_texts

# save
with open(f'processed/strec/{map_name_in_strec}/fully_processed_labels.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

865 labels.
604 labels.
502 labels.
465 labels.
450 labels.
443 labels.
441 labels.
Sequence Recovery completed with 441 labels.


#### C. Our Method - New, but why isn't it reducing as much as old?

In [3]:
# load in seq rec prepared
df = pickle.load(open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'rb'))

bezier_costs = df['bezier_scores'].tolist()
font_similarities = df['font_scores'].tolist()
neighbours = df['neighbours'].tolist()
labels = df['labels'].tolist()
indices, labels, bezier_costs, font_similarities, neighbours, text_lists = slsr.sl_seq_req_wrapper(labels, bezier_costs, font_similarities, neighbours, 0, 1.5)
df = pd.DataFrame({"labels":labels})

# save
with open(f'processed/strec/{map_name_in_strec}/fully_processed_labels_altslsr.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)


865 labels.
509 labels.
482 labels.
Retained 482.


# 4: Evaluation

In [4]:
from PIL import Image, ImageFile
import pandas as pd
from itertools import combinations
import scipy
import numpy as np
import importlib 

import Evaluation
import ExtractHandling
importlib.reload(Evaluation)
%load_ext autoreload

## 4.1: Isolate crops to be used for evaluation

In [33]:

def visualize_crop(map_name_in_strec, raw_or_spotter, left_x, right_x, top_y, bottom_y):
    if raw_or_spotter == "raw":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/raw.jpeg') 
    elif raw_or_spotter == "spotter_0":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_0.png')
    elif raw_or_spotter == "spotter_1":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_1.png')
    elif raw_or_spotter == "spotter_2":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_2.png')
    elif raw_or_spotter == "all":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_all_layers.png')
    elif raw_or_spotter == "rectified":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_all_layers_rectified.png')
    width, height = map_img.size
    print("full map is " + str(width) + " pixels wide by " + str(height) + " pixels high.\n displaying crop:")
    display(map_img.crop((left_x, top_y, right_x, bottom_y, )))

left_x = 2475
right_x = 3550
top_y = 4820
bottom_y = 5850

#visualize_crop("kiepert_1845", "all", left_x, right_x, top_y, bottom_y)

## 4.2 Precision and Recall: 1:1 Matching on Geometry, then IoU

In [5]:
kiepert_gt_patches = [[1750, 3750, 4775, 6200], [2250, 4050, 6050, 7500]] #,[2475, 3550, 4820, 5850]]
saunders_gt_patches = [[2350, 3850, 1750, 3250], [6450, 7500, 2200, 3250], [5400, 6400, 4500, 5500], [7650, 8650, 5400, 6400], [7650, 8650, 3150, 4150]] #
vandevelde_gt_patches = [[2850, 5250, 1450, 3850]]

# Gimme them numbers :)

kname = "kiepert_1845"
vname = "vandevelde_1846"
sname = "saunders_1874"

multiline_handling = "components" # "largest" for multiline gt

#Evaluation.prec_rec_wrapper(kname, multiline_handling, kiepert_gt_patches)
#Evaluation.prec_rec_wrapper(sname, multiline_handling, saunders_gt_patches)
Evaluation.prec_rec_wrapper(vname, multiline_handling, vandevelde_gt_patches)


vandevelde_1846 baseline (gt = components)

Avg of Geographic Precision: 0.10768564771776375
Avg of Text Precision: 0.11568854680573216
Avg of Geographic Recall: 0.5109340306608791
Avg of Text Recall: 0.548905232716559

vandevelde_1846 pyramid - subword dedup, nested word flattening (gt = components)

Avg of Geographic Precision: 0.29380799413381703
Avg of Text Precision: 0.29109991325831946
Avg of Geographic Recall: 0.5532342017200597
Avg of Text Recall: 0.5481349430502399

vandevelde_1846 pyramid - subword dedup, nested word flattening, our sequence recovery (gt = components)

Avg of Geographic Precision: 0.5153360389487641
Avg of Text Precision: 0.5968818765592219
Avg of Geographic Recall: 0.3234555989146498
Avg of Text Recall: 0.3746386246488733

vandevelde_1846 pyramid - subword dedup, nested word flattening, our sequence recovery alt (gt = components)

Avg of Geographic Precision: 0.5239873569999334
Avg of Text Precision: 0.5843954148984797
Avg of Geographic Recall: 0.6131766943

## 5: Plot and Save results

In [6]:
import shapely as sh
save_map_name_in_strec = 'vandevelde_1846' # 'kiepert_1845', 'saunders_1874', 'vandevelde_1846'

# Plot the final image
Evaluation.plot_recovered_seq(save_map_name_in_strec, "methods_1_2_3")
#Evaluation.plot_recovered_seq(save_map_name_in_strec, 'methods_1_2_r', '(rumsey)')
Evaluation.plot_recovered_seq(save_map_name_in_strec, 'methods_1_2_3a', '(alt)')

In [34]:
# Save final results into a json file
spotter_labels_full = ExtractHandling.load_processed_labels(save_map_name_in_strec, "methods_1_2_3")
polygon_list = spotter_labels_full['label_polygons'].tolist()
text_list = spotter_labels_full['annotation'].tolist()

polygons_json = []
for poly, text in zip(polygon_list, text_list):
    dict = {'polygon_x': [], 'polygon_y': [], 'text': text}
    if isinstance(poly, sh.geometry.polygon.Polygon):
        dict['polygon_x'] = list(poly.exterior.coords.xy[0])
        dict['polygon_y'] = list(poly.exterior.coords.xy[1])
            
    elif isinstance(poly, sh.geometry.multipolygon.MultiPolygon):
        for p in poly.geoms: # kaede added .geoms - package version differences
            polygon_x = p.exterior.coords.xy[0]
            polygon_y = p.exterior.coords.xy[1]

            for x, y in zip(polygon_x, polygon_y):
                dict['polygon_x'].append(x)
                dict['polygon_y'].append(y)

    polygons_json.append(dict)

with open(f'processed/strec/{save_map_name_in_strec}/final.json', 'w') as f:
    json.dump(polygons_json, f)